In [1]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2

In [2]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [3]:
image=cv2.imread(".//images//test_03.png")
# cv2.imshow("orignal image",image)
# cv2.waitKey(0)

In [4]:
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blurred=cv2.GaussianBlur(gray,(5,5),0)
edged=cv2.Canny(blurred,75,100)
# cv2.imshow("orignal image",edged)
# cv2.waitKey(0)

In [5]:
cnts=cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)
print(len(cnts))
docCnt=None
if (len(cnts)>0):
    cnts=sorted(cnts,key=cv2.contourArea,reverse=True)
    for c in cnts:
        peri=cv2.arcLength(c,True)
        approx=cv2.approxPolyDP(c,0.02*peri,True)
        if(len(approx)==4):
            docCnt=approx
            break

# cv2.drawContours(image,[docCnt],-1,(0,255,0),2)
# cv2.imshow("image",image)
# cv2.waitKey(0)

1


In [6]:
#bird eye view
paper=four_point_transform(image,docCnt.reshape(4,2))
wrapped=four_point_transform(gray,docCnt.reshape(4,2))
# cv2.imshow("image",paper)
# cv2.waitKey(0)
# cv2.imshow("image",wrapped)
# cv2.waitKey(0)

In [7]:
#binarization of image
thresh=cv2.threshold(wrapped,0,255,
                     cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
# cv2.imshow("image",thresh)
# cv2.waitKey(0)

In [8]:
cnts=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,
                     cv2.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)
ques=[]
for c in cnts:
    (x,y,w,h)=cv2.boundingRect(c)
    ar=w/float(h)
    
    if(w>=20 and h>=20 and ar>=0.9 and ar<=1.1):
        ques.append(c)
#         cv2.drawContours(paper,[c],-1,(0,255,0),2)
#         cv2.imshow("image",paper)
#         cv2.waitKey(0)

In [9]:
ques=contours.sort_contours(ques,method="top-to-bottom")[0]
correct=0

In [10]:
for (q, i) in enumerate(np.arange(0, len(ques), 5)):
    # sort the contours for the current question from
    # left to right, then initialize the index of the
    # bubbled answer
    cnts = contours.sort_contours(ques[i:i + 5])[0]
    bubbled = None

    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current
        # "bubble" for the question
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the
        # bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        
#         cv2.imshow("image",mask)
#         cv2.waitKey(0)

        # if the current total has a larger number of total
        # non-zero pixels, then we are examining the currently
        # bubbled-in answer
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    # initialize the contour color and the index of the
    # *correct* answer
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    # check to see if the bubbled answer is correct
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1

    # draw the outline of the correct answer on the test
    cv2.drawContours(paper, [cnts[k]], -1, color, 3)

In [11]:
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)

[INFO] score: 60.00%


-1

In [12]:
#sobel operator is good for edge detection